In [1]:
import numpy as np
import itertools
import os, sys

import torch
import torch.nn as nn
import torch.nn.functional  as F

from progressBar import ProgressBar

## Get the dataset

In [2]:
from torch.utils import data

In [3]:
class Dataset(data.Dataset):
    def __init__(self, names = 'SM', 
                 template='../data/20190621_5part_PtOrder/{}_lepFilter_13TeV.npy'):
        if names == 'SM':
            self.SM_names = ['Wlnu', 'qcd', 'ttbar']
            names = self.SM_names
        self.names = names
        
        for i, n in enumerate(names):
            ins = np.load(template.format(n)).astype(np.float32)
            out = i*np.ones(ins.shape[0]).astype(np.int8)
            
            print(n, ':', str(ins.shape[0]))
            
            if i == 0:
                self.inputs = ins
                self.outputs = out
            else:
                self.inputs = np.concatenate((self.inputs, ins))
                self.outputs = np.concatenate((self.outputs, out))
    
    def __len__(self):
        return self.inputs.shape[0]
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

In [4]:
dataset = {}
# Use to do validation split
# torch.utils.data.random_split(dataset, lengths)
dataset['train'] = Dataset()

Wlnu : 10000
qcd : 10000
ttbar : 10000


## Define the model

In [ ]:
class AEGraphNet(nn.Module):
    def __init__(self, N_nodes, N_features, dim_hidden, De, Do, dim_latent, verbose = False):
        super(ClfGraphNet, self).__init__()
        self.verbose = verbose
        
        self.p = N_features
        self.No = N_nodes
        self.De = De
        self.Do = Do
        
        self.allo
        self.Rr, self.Rs = self.buildEdgesMatrixes()
        self.fr = self.build_fr(dim_hidden)
        self.fo = self.build_fo(dim_hidden)
        self.PhiC = self.build_PhiC(dim_latent)
        
        self.onGPU = False
        
        
    def buildEdgesMatrixes(self):
        ### Assume fully connected graph
        Ne = self.No * (self.No - 1)
        self.Ne = Ne
        Rr = torch.zeros(self.No, Ne)
        Rs = torch.zeros(self.No, Ne)
        receiver_sender_list = [i for i in itertools.product(range(self.No), range(self.No)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            Rr[r, i] = 1
            Rs[s, i] = 1
        return Rr, Rs
            
    def build_fr(self, dim_hidden):
        fr = nn.Sequential(
                            nn.Linear(2 * self.p, dim_hidden),
                            nn.ReLU(),
                            nn.Linear(dim_hidden, int(dim_hidden/2)),
                            nn.ReLU(),
                            nn.Linear(int(dim_hidden/2), self.De),
                            nn.ReLU(),
                          )
        return fr
        
    def build_fo(self, dim_hidden):
        fo = nn.Sequential(
                            nn.Linear(self.p + self.De, dim_hidden),
                            nn.ReLU(),
                            nn.Linear(dim_hidden, int(dim_hidden/2)),
                            nn.ReLU(),
                            nn.Linear(int(dim_hidden/2), self.Do),
                            nn.ReLU(),
                          )    
        return fo

    def build_PhiC(self, dim_output):
        self.PhiC_layer = nn.Linear(self.Do, dim_output)
        
        def PhiC(x):
            return F.sigmoid(self.PhiC_layer(x))
        
        return PhiC
         
    def INlayer(self, x):
        x = torch.transpose(x, 1, 2).contiguous()
        Orr = torch.matmul(x, self.Rr)
        Ors = torch.matmul(x, self.Rs)
        B = torch.cat([Orr, Ors], dim=1)
        
        ### First MLP ###
        E = self.fr(B.view(-1, 2 * self.p))
        del B
        
        E = E.view(-1, self.Ne, self.De)
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = torch.matmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        
        C = torch.cat([x, Ebar], 1)
        del Ebar
        
        C = torch.transpose(C, 1, 2).contiguous()
        O = self.fo(C.view(-1, self.p + self.De))
        del C
        O = torch.sigmoid(O.view(-1, self.No, self.Do))
        return O
        
    def forward(self, x):
        O = self.INlayer(x)
        
        # now sum over No nodes to obtain the Do latent quantities
        O = torch.sum(O, 1)
        y = self.PhiC(O)
        return y
    
    def useGPU(self):
        if torch.cuda.is_available():
            print('Current device: {} ({} available)'.format(torch.cuda.current_device(), 
                                                             torch.cuda.device_count()))
            
            for o in self.__dict__.values():
                if o.__class__ == torch.Tensor:
                    o.cuda()
            
            self.onGPU = True
        else: 
            print('GPU not available')